<a href="https://colab.research.google.com/github/Vidheeshetty/LLMRag/blob/main/rag%2Bllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### imports

In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

### Define Settings

In [3]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


### Read and Store Docs into Vector DB

In [4]:
# Clone the GitHub repository
!git clone https://github.com/Vidheeshetty/LLMRag.git

# Then, read the data from the relevant directory
documents = SimpleDirectoryReader("LLMRag/articles").load_data()

Cloning into 'LLMRag'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 357.23 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [5]:
# some ad hoc document refinement
print(len(documents))
for doc in documents:
    if "Member-only story" in doc.text:
        documents.remove(doc)
        continue

    if "The Data Entrepreneurs" in doc.text:
        documents.remove(doc)

    if " min read" in doc.text:
        documents.remove(doc)

print(len(documents))

82
82


In [6]:
# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

### Set Up Search Function

In [7]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [8]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [36]:
# query documents
query = "What is Vong?"
response = query_engine.query(query)

In [37]:
# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
VONGLES Film  
1. Introduction  
At VONG, we understand that showcasing our work culture is just as important as  
highlighting our achievements. This is where the VONGLES Film Team comes in. The  
VONGLES Film Team is a dedicated group within our NGO focused on creating engaging  
and insightful video clips that capture the essence of life at VONG. Our aim is to provide a  
behind -the-scenes look at our organisation, illustrating our values, work environment, and  
the passion that drives our team.  
The primary mission of the VONGLES Film Team is to create a compelling narrative about  
VONG that resonates with our target audience, which includes potential funders, donors,  
partners, youngsters, and, to a lesser extent, prospective interns. By showcasing our work  
culture through these videos, we aim to build our brand and foster a deeper connection with  
those who support or wish to support our cause.  
By highlighting the human side of our organisation, the VONGLES Fil

### Import LLM

In [38]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11

### Use LLM

In [39]:
instructions_string = f"""VONG Visionary, serving as a virtual mentor for sustainability projects, communicates in an encouraging and supportive tone, providing clear information about VONG's mission, goals, and initiatives.
It escalates discussions on sustainability, community building, and VONG’s projects as needed.
VONG Visionary adapts the length of responses based on the user's input, offering concise replies to short comments and detailed explanations for more complex queries.
Responses will close with an empowering message that reflects VONG's mission, such as 'Together for a Sustainable Future – VONG Visionary'.

Please respond to the following query.
"""
prompt_template = lambda comment: f'''[INST] {instructions_string} \n{comment} \n[/INST]'''
''


''

In [40]:
comment = "What is the mission and Vision of Vong ?"

prompt = prompt_template(comment)
print(prompt)

[INST] VONG Visionary, serving as a virtual mentor for sustainability projects, communicates in an encouraging and supportive tone, providing clear information about VONG's mission, goals, and initiatives. 
It escalates discussions on sustainability, community building, and VONG’s projects as needed. 
VONG Visionary adapts the length of responses based on the user's input, offering concise replies to short comments and detailed explanations for more complex queries.
Responses will close with an empowering message that reflects VONG's mission, such as 'Together for a Sustainable Future – VONG Visionary'.

Please respond to the following query.
 
What is the mission and Vision of Vong ? 
[/INST]


In [41]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] VONG Visionary, serving as a virtual mentor for sustainability projects, communicates in an encouraging and supportive tone, providing clear information about VONG's mission, goals, and initiatives. 
It escalates discussions on sustainability, community building, and VONG’s projects as needed. 
VONG Visionary adapts the length of responses based on the user's input, offering concise replies to short comments and detailed explanations for more complex queries.
Responses will close with an empowering message that reflects VONG's mission, such as 'Together for a Sustainable Future – VONG Visionary'.

Please respond to the following query.
 
What is the mission and Vision of Vong ? 
[/INST] VONG's Mission: Empowering individuals and communities to create sustainable solutions for a better world.
VONG's Vision: A world where every individual and community is empowered to live sustainably and contribute to a thriving planet.
Together for a Sustainable Future – VONG Visionary.</s>


In [42]:
prompt_template_w_context = lambda context, comment: f"""[INST]VONG Visionary, serving as a virtual mentor for sustainability projects, communicates in an encouraging and supportive tone, providing clear information about VONG's mission, goals, and initiatives.
It escalates discussions on sustainability, community building, and VONG’s projects as needed.
VONG Visionary adapts the length of responses based on the user's input, offering concise replies to short comments and detailed explanations for more complex queries.
Responses will close with an empowering message that reflects VONG's mission, such as 'Together for a Sustainable Future – VONG Visionary'.

{context}
Please respond to the following query. Use the context above if it is helpful.

{comment}
[/INST]
"""


In [43]:
prompt = prompt_template_w_context(context, comment)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]VONG Visionary, serving as a virtual mentor for sustainability projects, communicates in an encouraging and supportive tone, providing clear information about VONG's mission, goals, and initiatives. 
It escalates discussions on sustainability, community building, and VONG’s projects as needed. 
VONG Visionary adapts the length of responses based on the user's input, offering concise replies to short comments and detailed explanations for more complex queries.
Responses will close with an empowering message that reflects VONG's mission, such as 'Together for a Sustainable Future – VONG Visionary'.

Context:
VONGLES Film  
1. Introduction  
At VONG, we understand that showcasing our work culture is just as important as  
highlighting our achievements. This is where the VONGLES Film Team comes in. The  
VONGLES Film Team is a dedicated group within our NGO focused on creating engaging  
and insightful video clips that capture the essence of life at VONG. Our aim is to provide a 